In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers
import os
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk

In [2]:
# Data Loading and Preprocessing
def load_data(real_images_folder, cartoon_images_folder, image_size):
    real_files = os.listdir(real_images_folder)
    cartoon_files = os.listdir(cartoon_images_folder)
    
    # Ensure both lists are sorted for correspondence
    real_files.sort()
    cartoon_files.sort()
    
    # Ensure lists are of the same length or iterate up to the length of the shortest list
    min_length = min(len(real_files), len(cartoon_files))
    
    real_images = []
    cartoon_images = []
    
    for i in range(min_length):
        real_file = os.path.join(real_images_folder, real_files[i])
        cartoon_file = os.path.join(cartoon_images_folder, cartoon_files[i])
        
        real_image = Image.open(real_file).convert('RGB').resize((image_size, image_size))
        cartoon_image = Image.open(cartoon_file).convert('RGB').resize((image_size, image_size))
        
        real_images.append(real_image)
        cartoon_images.append(cartoon_image)
    
    return real_images, cartoon_images

def preprocess_images(image_list, image_size):
    # Convert a list of PIL images to a NumPy array
    image_arrays = [np.array(image) for image in image_list]
    # Stack arrays to form a batch and normalize pixel values to [0, 1]
    return np.stack(image_arrays, axis=0) / 255.0

In [3]:
# Generator Model
def build_generator(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, kernel_size=3, strides=1, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2DTranspose(32, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    outputs = layers.Conv2D(3, kernel_size=3, strides=1, padding='same', activation='sigmoid')(x)
    return models.Model(inputs, outputs)

# Discriminator Model
def build_discriminator(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = layers.LeakyReLU(alpha=0.2)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, outputs)

In [4]:
# Define CycleGAN Model
def cycle_gan_model(input_shape):
    # Initialize models
    generator_g = build_generator(input_shape)
    generator_f = build_generator(input_shape)
    discriminator_x = build_discriminator(input_shape)
    discriminator_y = build_discriminator(input_shape)
    
    # Compile discriminators
    bce_loss = losses.BinaryCrossentropy()  # Remove from_logits=True
    discriminator_x.compile(loss=bce_loss, optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5))
    discriminator_y.compile(loss=bce_loss, optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5))
    
    # Freeze discriminators for adversarial training
    discriminator_x.trainable = False
    discriminator_y.trainable = False
    
    # CycleGAN model
    input_real = layers.Input(shape=input_shape)
    input_cartoon = layers.Input(shape=input_shape)
    
    # Generators
    fake_cartoon = generator_g(input_real)
    fake_real = generator_f(input_cartoon)
    recon_real = generator_f(fake_cartoon)
    recon_cartoon = generator_g(fake_real)
    
    # Discriminators
    valid_x = discriminator_x(fake_real)
    valid_y = discriminator_y(fake_cartoon)
    
    # CycleGAN model
    cycle_gan = models.Model(inputs=[input_real, input_cartoon],
                             outputs=[valid_x, valid_y, recon_real, recon_cartoon])
    
    # Cycle consistency loss
    def cycle_loss(y_true, y_pred):
        return tf.reduce_mean(tf.abs(y_true - y_pred))
    
    # Compile CycleGAN
    cycle_gan.compile(loss=[bce_loss, bce_loss, cycle_loss, cycle_loss],
                      loss_weights=[1, 1, 10, 10],
                      optimizer=optimizers.Adam(learning_rate=0.0002, beta_1=0.5))
    
    return cycle_gan, generator_g, generator_f, discriminator_x, discriminator_y

In [5]:
# Load data
real_images_folder = r'C:\Users\aruns\Downloads\realimages'
cartoon_images_folder = r'C:\Users\aruns\Downloads\selfie\trainB'

# Define input shape
image_size = 128  # Reduced image size to reduce memory usage
input_shape = (image_size, image_size, 3)
batch_size = 8  # Reduced batch size to reduce memory usage

real_images, cartoon_images = load_data(real_images_folder, cartoon_images_folder, image_size)

In [6]:
# Balance the datasets
min_dataset_size = min(len(real_images), len(cartoon_images))
real_images = real_images[:min_dataset_size]
cartoon_images = cartoon_images[:min_dataset_size]

In [7]:
# Initialize CycleGAN
cycle_gan, generator_g, generator_f, discriminator_x, discriminator_y = cycle_gan_model(input_shape)

# Training Loop (Example: Run for a few epochs, adjust as needed)
num_epochs = 70  # Set to your desired number of epochs
num_batches = min(len(real_images), len(cartoon_images)) // batch_size

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}", flush=True)
    
    for batch_index in range(num_batches):
        start_index = batch_index * batch_size
        end_index = start_index + batch_size
        real_batch = preprocess_images(real_images[start_index:end_index], image_size)
        cartoon_batch = preprocess_images(cartoon_images[start_index:end_index], image_size)

        # Train Discriminators
        fake_cartoon_images = generator_g.predict(real_batch)
        fake_real_images = generator_f.predict(cartoon_batch)
        
        d_loss_real_x = discriminator_x.train_on_batch(real_batch, tf.ones((batch_size, 1)))
        d_loss_fake_x = discriminator_x.train_on_batch(fake_real_images, tf.zeros((batch_size, 1)))
        d_loss_x = 0.5 * (d_loss_real_x + d_loss_fake_x)
        
        d_loss_real_y = discriminator_y.train_on_batch(cartoon_batch, tf.ones((batch_size, 1)))
        d_loss_fake_y = discriminator_y.train_on_batch(fake_cartoon_images, tf.zeros((batch_size, 1)))
        d_loss_y = 0.5 * (d_loss_real_y + d_loss_fake_y)

        # Train Generators with cycle consistency
        g_loss = cycle_gan.train_on_batch([real_batch, cartoon_batch], 
                                          [tf.ones((batch_size, 1)), tf.ones((batch_size, 1)), 
                                           real_batch, cartoon_batch])

        print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_index+1}/{num_batches}, "
              f"D Loss X: {d_loss_x}, D Loss Y: {d_loss_y}, G Loss: {g_loss}")

Epoch 1/70
Epoch 1/70, Batch 1/425, D Loss X: 0.7921499311923981, D Loss Y: 0.8048078119754791, G Loss: [6.042349815368652, 0.5987046957015991, 0.5993884801864624, 0.24854826927185059, 0.2358773946762085]
Epoch 1/70, Batch 2/425, D Loss X: 0.7155812382698059, D Loss Y: 0.7214262783527374, G Loss: [5.877842426300049, 0.6318463087081909, 0.6234858632087708, 0.23390300571918488, 0.2283480167388916]
Epoch 1/70, Batch 3/425, D Loss X: 0.708897054195404, D Loss Y: 0.7090091407299042, G Loss: [6.029308319091797, 0.6809112429618835, 0.638086199760437, 0.237621009349823, 0.23341004550457]
Epoch 1/70, Batch 4/425, D Loss X: 0.6998833417892456, D Loss Y: 0.6982333660125732, G Loss: [6.0790228843688965, 0.7235721349716187, 0.6576719284057617, 0.23215647041797638, 0.23762142658233643]
Epoch 1/70, Batch 5/425, D Loss X: 0.6947148740291595, D Loss Y: 0.7000882029533386, G Loss: [6.308923721313477, 0.7365727424621582, 0.6710871458053589, 0.2657863199710846, 0.22434008121490479]
Epoch 1/70, Batch 6/425

In [8]:
# Save the generator models
generator_g.save('cycle_gan_generator_g2.h5')
generator_f.save('cycle_gan_generator_f2.h5')

In [9]:
# Load the saved generator model (Example: Loading the generator)
generator_g = tf.keras.models.load_model('cycle_gan_generator_g2.h5')
generator_f = tf.keras.models.load_model('cycle_gan_generator_f2.h5')

In [ ]:
# Function to preprocess and resize the real image
def preprocess_and_resize_image(image_path, target_size=(128, 128)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize the image
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return np.expand_dims(image, axis=0)  # Add batch dimension

In [ ]:
# Function to convert real image to cartoon image using the generator model
def convert_to_cartoon(real_image):
    cartoon_image = generator_g.predict(real_image)
    return cartoon_image

In [ ]:
def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        # Preprocess and resize the uploaded image
        unseen_real_image = preprocess_and_resize_image(file_path)
        # Convert the resized image to a cartoon image
        cartoon_image = convert_to_cartoon(unseen_real_image)
        # Display the original and cartoon images
        original_image = Image.open(file_path)
        original_image.thumbnail((400, 400))
        cartoon_image = Image.fromarray((cartoon_image[0] * 255).astype(np.uint8))
        cartoon_image.thumbnail((400, 400))
        original_image_tk = ImageTk.PhotoImage(original_image)
        cartoon_image_tk = ImageTk.PhotoImage(cartoon_image)
        original_label.config(image=original_image_tk)
        original_label.image = original_image_tk
        cartoon_label.config(image=cartoon_image_tk)
        cartoon_label.image = cartoon_image_tk
        # Set size of cartoon label to match original label
        cartoon_label.config(width=original_image_tk.width(), height=original_image_tk.height())

In [ ]:
# Create a Tkinter window
root = tk.Tk()
root.title("Image Cartooning System")

# Create a frame
frame = tk.Frame(root)
frame.pack(padx=10, pady=10)

# Create labels for original and cartoon images
original_label = tk.Label(frame)
original_label.pack(side="left", padx=10, pady=10)
cartoon_label = tk.Label(frame)
cartoon_label.pack(side="right", padx=10, pady=10)

# Create a button to open an image
open_button = tk.Button(root, text="Open Image", command=open_image)
open_button.pack(pady=10)

root.mainloop()